In [1]:
#importing libraries

import numpy as np
import pandas as pd
import os
import errno
import imageio

In [2]:
# initialization
image_height = 48
image_width = 48
window_size = 24
window_step = 6
SAVE_IMAGES = True
SELECTED_LABELS = [0,1,2,3,4,5,6]
IMAGES_PER_LABEL = 500
OUTPUT_FOLDER_NAME = "datasets"

In [3]:
def data_download():
    # loading Dlib predictor and preparing arrays:
    print( "preparing")

    #predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
    original_labels = [0, 1, 2, 3, 4, 5, 6]
    new_labels = list(set(original_labels) & set(SELECTED_LABELS))
    nb_images_per_label = list(np.zeros(len(new_labels), 'uint8'))
    try:
        os.makedirs(OUTPUT_FOLDER_NAME)
    except OSError as e:
        if e.errno == errno.EEXIST and os.path.isdir(OUTPUT_FOLDER_NAME):
            pass
        else:
            raise
            
    
    print( "importing csv file")
    
    data = pd.read_csv('fer2013.csv')
    
    for category in data['Usage'].unique():
        print( "converting set: " + category + "...")
        # create folder
        if not os.path.exists(category):
            try:
                os.makedirs(OUTPUT_FOLDER_NAME + '/' + category)
            except OSError as e:
                if e.errno == errno.EEXIST and os.path.isdir(OUTPUT_FOLDER_NAME):
                   pass
                else:
                    raise
        
        # get samples and labels of the actual category
        category_data = data[data['Usage'] == category]
        samples = category_data['pixels'].values
        print("Here is length of samples: " + str(len(samples)))
        labels = category_data['emotion'].values
    
        # get images and extract features
        images = []
        labels_list = []
        landmarks = []
        hog_features = []
        hog_images = []
        for i in range(len(samples)):
            try:
                if labels[i] in SELECTED_LABELS: 
                    image = np.fromstring(samples[i], dtype=int, sep=" ").reshape((image_height, image_width))
                    
                    # Convert the image to uint8
                    image = np.uint8(image)
                    
                    images.append(image)
                    output_path = os.path.join(OUTPUT_FOLDER_NAME, category, str(i + 1) + '.jpg')
                    imageio.imwrite(output_path, image)
                    #imageio.imwrite(OUTPUT_FOLDER_NAME + '/' + category + '/' + str(i+1) + '.jpg', image)
                    
            except Exception as e:
                print( "error in image: " + str(i) + " - " + str(e))
    
        np.save(OUTPUT_FOLDER_NAME + '/' + category + '/images.npy', images)
        #prints out file names in each folder of category
        """output_folder = "datasets"
        file_names = sorted(os.listdir(os.path.join(output_folder, category)))
        for file_name in file_names:
            print(file_name)"""
        
#Deletes Cleans up "dataset" folder so that data_dowland method can re-make images        
#import shutil
#shutil.rmtree('datasets', ignore_errors=True)